## Basic Pipeline Example

This notebook shows how to use basic functions of the Cortex Python SDK pipeline. 
In this example, see how to modify or enrich datasets to make them suitable for training or modeling. 
Data is modified in a sequential series of steps.

In [ ]:
# Import Cortex and other required libraries
import math
import numpy as np
import pandas as pd
from cortex import Cortex

# Create a Builder instance
cortex = Cortex.local()
builder = cortex.builder()

In the next step, create a data set and populate it from a comma separated values file. A pipeline operates on a dataset.

In [ ]:
dataset_name_ff = input("namespace/dataset name")

data_set = builder.dataset(dataset_name_ff).title('Forest Fire Data')\
    .from_csv('data/ff.sample.csv').build()
# Create a pandas DataFrame to view the last few lines of the dataset
data_frame = data_set.as_pandas()
data_frame.tail(20)

**validate types**

We need to make sure that the types in the columns are what we are expecting.

In [ ]:
print(data_frame.dtypes)

A dataset can have one or more named pipelines. Each pipeline is a chain of Python functions that transform the dataset.  In the next step, create a pipeline named "prep".

In [ ]:
pipeline = data_set.pipeline('prep') # create or retrieve the pipline named 'prep'
pipeline.reset() # removes any previous steps or context for this pipeline

One pipeline step can be used to add a new column.

This [dataset](http://piano.dsi.uminho.pt/~pcortez/fires.pdf) uses components from the Fire Weather Index to make predictions. One element, the Build Up Index (BUI) is based on a relation of two other columns and is omitted. This step adds that missing element. 

In [ ]:
def normalize(pipeline, df):
    cols = ["DMC", "DC", "area", "X", "Y", "FFMC", "temp"]
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

pipeline.add_step(normalize)

In [ ]:
def add_bui(pipeline, df):
    df['BUI'] = (0.8 * df['DMC'] * df['DC'])/(df['DMC'] + 0.4 * df['DC'])

pipeline.add_step(add_bui)

In the preceeding code, the pipeline step functions require a pipeline and a dataframe parameter. The [pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe) provides a rich set of functions for operating on table data.

A pipeline step may be used to modify a column.

The dataset's documentation says that the last column, __area__, is skewed towards zero and should be adjusted logarithmically "to improve regression results for right-skewed targets".

In [ ]:
def fix_area(pipeline, df):
    df['area'] = df['area'].map(lambda a: math.log1p(a))
    
pipeline.add_step(fix_area)

### Running the Pipeline
After all the steps are added, you can call `run` on the pipeline. This invokes each of the steps in order and returns a transformed DataFrame instance.

In [ ]:
data_frame = pipeline.run(data_frame)

You can verify what your pipeline did here:

In [ ]:
data_frame.dtypes

Here you can visualize your data

In [ ]:
def draw_graphs(data_set, data_frame, attribute):

    v = data_set.visuals(data_frame)
    v.show_corr_heatmap()
    v.show_corr(attribute)
    v.show_corr_pairs(attribute)
    v.show_dist(attribute)
    v.show_probplot(attribute)

In [ ]:
draw_graphs(data_set, data_frame, 'area')

Lets save our pipeline for later use

In [ ]:
pipeline.to_camel()

In [ ]:
data_set.save()

**Break point**

We have added metadata to our local sample dataset, now we want to use this pipeline on the full dataset and move it to cortex remote

In [ ]:
# Create a non-local Builder instance
from cortex.builder.dataset_builder import DatasetBuilder
cortex = Cortex.client()
builder = cortex.builder()

In [ ]:
# to help with making this dataset distinct in class we will use an input generator here for the dataset name.  
# This variable will be stored throughout this example.
dataset_forestfire = input("namespace/dataset name")
    


#csv_example_data_set = csv_data_set_builder.from_csv('./data/sample_large.csv').build()

We'll take a look at the pipeline camel spec here



In [ ]:
pipeline.to_camel()

In [ ]:
data_set = builder.dataset(dataset_forestfire).title('Forest Fire Data')\
    .from_csv('data/forestfires.csv').build()
# Create a pandas DataFrame to view the last few lines of the dataset
data_frame = data_set.as_pandas()
data_frame.tail(20)

In [ ]:
print(pipeline)

In [ ]:
remote_pipeline = data_set.pipeline('prep') # create or retrieve the pipline named 'prep'
remote_pipeline.from_pipeline(pipeline)
#pipeline.reset() # removes any previous steps or context for this pipeline

In [ ]:
remote_pipeline.run(data_frame)